DATA LOADING

In [ ]:
import pandas as pd
import numpy as np

###############################################################################
# 1) Define File Paths for Each Country
###############################################################################
datasets = {
    "US": "/Users/thunguyen/Desktop/ThuThu/processed datasets/Extracting datasets/United States_Indicators_Wide.csv",
    "PL": "/Users/thunguyen/Desktop/ThuThu/processed datasets/Extracting datasets/Poland_Indicators_Wide.csv",
    "NL": "/Users/thunguyen/Desktop/ThuThu/processed datasets/Extracting datasets/Netherlands_Indicators_Wide.csv",
    "FR": "/Users/thunguyen/Desktop/ThuThu/processed datasets/Extracting datasets/France_Indicators_Wide.csv",
    "DE": "/Users/thunguyen/Desktop/ThuThu/processed datasets/Extracting datasets/Germany_Indicators_Wide.csv",
    "CZ": "/Users/thunguyen/Desktop/ThuThu/processed datasets/Extracting datasets/Czech Republic_Indicators_Wide.csv"
}

###############################################################################
# 2) Helper: Min–Max Normalize a Pandas Series
###############################################################################
def minmax_normalize(series):
    """Scales the data in `series` to [0, 1]."""
    min_val = series.min()
    max_val = series.max()
    if min_val == max_val:
        # If all values are the same, set them to 0.0 to avoid division by zero.
        return series.apply(lambda x: 0.0)
    return (series - min_val) / (max_val - min_val)

###############################################################################
# 3) Create Financial Stress Index Function
###############################################################################
def create_financial_stress_index(df):
    """
    This function:
      - Min–max normalizes the five key columns:
          1) Personal Savings
          2) Banks Balance Sheet
          3) Bank Lending Rate
          4) Consumer Confidence
          5) Business Confidence
      - Constructs a 'Financial_Stress' column with the sign logic:
          Financial_Stress = -CC - BC + BLR - PS - BBS
        i.e., higher Confidence or Savings or Banks Balance => lower stress,
              higher Lending Rate => more stress.
    """

    required_cols = [
        "Personal Savings",
        "Banks Balance Sheet",
        "Bank Lending Rate",
        "Consumer Confidence",
        "Business Confidence"
    ]

    # 1) Check that each required column exists, then min-max normalize
    for col in required_cols:
        if col not in df.columns:
            print(f"Warning: '{col}' not found in DataFrame. Creating NaN column.")
            df[col] = np.nan
        df[col + "_Norm"] = minmax_normalize(df[col])

    # 2) Build the stress index from the normalized columns
    ps_col = "Personal Savings_Norm"
    bbs_col = "Banks Balance Sheet_Norm"
    blr_col = "Bank Lending Rate_Norm"
    cc_col = "Consumer Confidence_Norm"
    bc_col = "Business Confidence_Norm"

    if all(x in df.columns for x in [ps_col, bbs_col, blr_col, cc_col, bc_col]):
        # Negative for Confidence, Savings, Banks Balance => reduce stress
        # Positive for Lending Rate => increase stress
        df["Financial_Stress"] = (
            - df[cc_col]     # higher consumer confidence => lower stress
            - df[bc_col]     # higher business confidence => lower stress
            + df[blr_col]    # higher lending rate => more stress
            - df[ps_col]     # higher personal savings => lower stress
            - df[bbs_col]    # larger banks balance => lower stress
        )
    else:
        df["Financial_Stress"] = np.nan

    return df

###############################################################################
# 4) Read Each CSV, Create Stress Index, Collect Data
###############################################################################
data_frames = {}
for country, file_path in datasets.items():
    # Load the dataset
    df = pd.read_csv(file_path)

    # Create the stress index
    df = create_financial_stress_index(df)

    # Store back into dictionary
    data_frames[country] = df
    print(f"{country} - columns: {df.columns.tolist()}\n")

###############################################################################
# 5) Merge All Countries by "Quarter"
###############################################################################
# We'll select "Quarter" + "Financial_Stress" from each DF, rename "Financial_Stress" -> country code,
# then do an outer join so we keep all quarters from all countries (filling missing with NaN).
merged_list = []
for country, df in data_frames.items():
    if "Quarter" not in df.columns or "Financial_Stress" not in df.columns:
        print(f"Warning: {country} missing 'Quarter' or 'Financial_Stress' column. Skipping.")
        continue
    temp = df[["Quarter", "Financial_Stress"]].copy()
    temp = temp.set_index("Quarter").rename(columns={"Financial_Stress": country})
    merged_list.append(temp)

# Outer join across all countries
merged_data = pd.concat(merged_list, axis=1, join="outer")

# (Optional) Drop rows that are all NaN or do partial fill
merged_data.dropna(how="all", inplace=True)

print("\n=== Merged Data (first few rows) ===")
print(merged_data.head(10))

# At this point, `merged_data` has one row per Quarter, with each country's stress index as columns.
# Next steps (e.g., stationarity checks, VAR, etc.) can follow as in your original approach.

CORRELATION AND ADJACENCY MATRIX

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Define sector mapping
sector_map = {
    "Household": ['Consumer Confidence', 'Personal Savings'],
    "Financial": ['Bank Lending Rate'],
    "Non-Financial": ['Business Confidence']
}

for sector, indicators in sector_map.items():
    merged_df = pd.DataFrame()

    for country in countries:
        df = pd.read_csv(datasets[country], parse_dates=['Quarter'])
        df = df.set_index('Quarter')
        
        # Only keep indicators for this sector
        available = [ind for ind in indicators if ind in df.columns]
        df = df[available].copy()
        df.columns = [f"{country} | {col}" for col in df.columns]

        merged_df = pd.concat([merged_df, df], axis=1)

    # Align time and clean
    merged_df = merged_df.dropna()

    # Compute correlation matrix
    corr_matrix = merged_df.corr()

    # Plot heatmap
    plt.figure(figsize=(14, 12))
    sns.heatmap(corr_matrix, cmap="coolwarm", center=0, linewidths=0.5, square=True)
    plt.title(f"{sector} Sector - Cross-Country Correlation Heatmap")
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()


VAR AND GRANGER-CAUSALITY TEST

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler

# statsmodels for VAR, ADF, Granger tests
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.api import VAR

# Standardize data for stability
scaler = StandardScaler()
merged_data_scaled = pd.DataFrame(scaler.fit_transform(merged_data), 
                                  columns=merged_data.columns, 
                                  index=merged_data.index)
# 1️⃣ Check Stationarity (Augmented Dickey-Fuller Test)
def check_stationarity(data):
    results = {}
    for col in data.columns:
        test_result = adfuller(data[col])
        results[col] = {"ADF Statistic": test_result[0], "p-value": test_result[1]}
    return pd.DataFrame(results).T

# Test for stationarity
stationarity_results = check_stationarity(merged_data_scaled)
print(stationarity_results)

# If any series is non-stationary (p > 0.05), difference it
if any(stationarity_results["p-value"] > 0.05):
    merged_data_scaled = merged_data_scaled.diff().dropna()

# 2️⃣ Fit VAR Model & Select Optimal Lag
var_model = VAR(merged_data_scaled)
selected_lag = var_model.select_order(maxlags=10)  # Adjust maxlags as needed
print(selected_lag.summary())

# Fit model with chosen lag
optimal_lag = selected_lag.aic  # Extract integer value
if np.isnan(optimal_lag):  # Handle potential NaN issue
    optimal_lag = selected_lag.bic  # Use BIC as a fallback

var_results = var_model.fit(optimal_lag)

# Plot the VAR results
var_results.plot()
plt.show()

# Compute FEVD for 10 periods
fevd = var_results.fevd(10)



# 3️⃣ Perform Granger Causality Tests
def granger_test(data, max_lag):
    causality_results = {}
    for col1 in data.columns:
        for col2 in data.columns:
            if col1 != col2:
                test_result = grangercausalitytests(data[[col1, col2]], max_lag, verbose=False)
                min_p_value = min([test_result[i+1][0]['ssr_chi2test'][1] for i in range(max_lag)])
                causality_results[(col1, col2)] = min_p_value
    return pd.DataFrame(causality_results.items(), columns=["Causal Relationship", "p-value"])

# Run Granger Causality
granger_results = granger_test(merged_data_scaled, max_lag=optimal_lag)
print(granger_results)

# 4️⃣ Compute Impulse Response Functions (IRF)
irf = var_results.irf(10)  # IRF over 10 periods

# 5️⃣ Plot IRFs
irf.plot(orth=True)
plt.show()
